In [3]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Activation


print(tf.__version__)


2.10.0


In [2]:
# Pandas dataframe to open the the CSV file and get the Descriptions data
# Load the data from the file
df = pd.read_csv('youtube.csv')
descriptions = df['description'].values
coupon_codes = df['coupon_code'].values

# Convert coupon codes into binary label
df["has_coupon_code"] = np.where(df["description"].str.contains("coupon|code"), 1, 0)

df.head()

NameError: name 'pd' is not defined

In [ ]:
# Split the data into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
vectorizer = TextVectorization(
    output_mode='int',
    max_tokens=1000,
    output_sequence_length=100)
text_ds = tf.data.Dataset.from_tensor_slices(train_df['descriptions']).batch(128)
vectorizer.adapt(text_ds)

In [ ]:
#Tokenization -> Text Vectorization (preprocessing layer that maps text feuatures to integer sequences)

# Make the output integer data type
#Max_tokens parameter is a parameter that sets the maximum number of tokens/unique words in the vocabulary after the model gets preprocessed
# Setting it to vocab size means you lose no vocbuary
vectorizer = TextVectorization(
    max_tokens=10000,
    output_mode="int",
    output_sequence_length=100
)
text_ds = tf.data.Dataset.from_tensor_slices(train_df["description"].values).batch(32)
vectorizer.adapt(text_ds)

#Vectorize the descriptions
train_text = vectorizer(train_df["description"])
test_text = vectorizer(test_df["description"])

In [ ]:
#Padding -> Adds 0's to the shorter sequences so all the input sequences stay the same
# Framworks like tensorflow expect all inputs to be the same
# Post padding is always nessecary and important because when it applies before 
# the data, the model will take a while to learn and make it inefficient
train_desc_padded = tf.keras.preprocessing.sequence.pad_sequences(train_desc, padding='post', maxlen=100)
test_desc_padded = tf.keras.preprocessing.sequence.pad_sequences(test_desc, padding='post', maxlen=100)

### RNN Architecture
- RNN - Reccurent Neural Network - makes predictions based on multiple inputs based on the problem 
- Sequential in terms of a network means -> input to output
- RNNs have feedback loops which set the network apart
    - Feedback oops -> (weightOfLayer * CurrentValue) + (y1*weigthOfFeedback)

In [1]:
#Build the Model
model = tf.keras.Sequential([
    # Mask_Zero=true removes all the assigned paddign tokens because the layer will learn to ignore the sequences that I padded
    tf.keras.layers.Embedding(input_dim=len(vectorizer.get_vocabulary()), output_dim=128, mask_zero=True),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

NameError: name 'Sequential' is not defined

### Flow of future data collection and traning
- Since there is no way to constantly labelize the data, Once the data is collected, the model will be manually labelized
- The model will have around 500 descriptions (video) -> maybe go up to a 1000
- From there we can make a NEW YOUTUBE API CALL -> this one is SPECIFICALLY FOR DATE relevant collection -> every week run
- Data from this new call will be run through the model and the codes will be recognized
- Then run the function to figure out all other information about the code(site, discount, creator) etc.
- Send all this info to client side in the form of a CSV file
- The entire process should be loop like

In [ ]:
#Compile and train
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train and Evaluate the model
history = model.fit(train_data, train_labels, epochs=10, batch_size=batch_size, validation_data=(test_data, test_labels))

# Evaluate the model
test_loss, test_acc = model.evaluate(test_data, test_labels)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

In [ ]:
# Test model
text = "Check out my new product and use the coupon code YOUTUBE10 for 10% off your purchase!"
sequence = tokenizer.texts_to_sequences([text])
padded_sequence = pad_sequences(sequence, maxlen=max_len)
prediction = model.predict(padded_sequence)
if prediction[0][0] > 0.5:
    coupon_code = text.split()[-1]
    print(f"Coupon code found: {coupon_code}")
else:
    print("No coupon code found.")